# Generate ERA5 forcing

This will consist of two parts in separate notebooks:
1. using a shapefile from Caravan
2. using your own shapefile!

ERA5 need 3 different inputs:
1. a shapefile
2. a time-window
3. a directory to save the data



## Caravan Shapefile

in eWaterCycle we provide access to the Caravan dataset, which contains all of the above data for all the catchments in the different Camels datasets. However, for two reasons, we don't use the rainfall and potential evaporation data from Caravan

1. This allows us to demonstrate how to generate forcing data for any model from ERA5 (working on ERA5Land) and from CMIP data
2. There is a known problem with the caravan evaporation data and we want to avoid using it here.

We thus only use the Caravan dataset to retrieve

- a shapefile of the region of interest, to be used as input for generating the forcing from ERA5
- the observational data of river discharge

Using the interactive maps at [eWaterCycle caravan map](https://www.ewatercycle.org/caravan-map/) one can easily retrieve the identifier of the catchment.

In [ ]:
# General python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Niceties
from rich import print

# General eWaterCycle
import ewatercycle
import ewatercycle.forcing

We will be using the same catchment from the Camels forcing from Caravan:

In [1]:
# The id of the camels region
camels_id = "camelsgb_33039"  # river: Bedford Ouse at Roxton, England

# The time-window of the experiment
experiment_start_date = "2000-08-01T00:00:00Z"
experiment_end_date = "2005-08-31T00:00:00Z"

In [ ]:
# The path of the Caravan data
forcing_path_caravan = Path.home() / "forcing" / camels_id / "caravan"
forcing_path_caravan.mkdir(exist_ok=True, parents=True)

Now if you ran the previous notebook (the one where you generate the forcing) just load the data, this saves time!

In [ ]:
'''Generating the camels forcing'''
camels_forcing = ewatercycle.forcing.sources['CaravanForcing'].generate(
    start_time=experiment_start_date,
    end_time=experiment_end_date,
    directory=forcing_path_caravan,
    basin_id=camels_id,
)

'''Loading the camels forcing'''
# camels_forcing = ewatercycle.forcing.sources['CaravanForcing'].load(directory=forcing_path_caravan)

### Extracting the shapefile

Now we can easily extract the shapefile

In [ ]:
shapefile = camels_forcing.shapefile

## ERA5 Forcing Generation

Now that we have the shapefile we can almost get the ERA5 forcing.
But first we need to define a new path.
And we need to pick what kind of forcing we want to generate, this depends on what model you want to use, currently these sources are supported:
1. LumpedMakkinkForcing
2. DistributedMakkinkForcing
3. DistributedUserForcing
4. GenericDistributedForcing
5. GenericLumpedForcing
6. HypeForcing
7. LisfloodForcing
8. LumpedUserForcing
9. MarrmotForcing
10. PCRGlobWBForcing
11. WflowForcing

More details can be found in the [documentation](https://ewatercycle.readthedocs.io/en/latest/user_guide/02_forcing.html). And also in the [models section](https://www.ewatercycle.org/getting-started/main/some_content/different_models.html).

For now we will use 'lumped Makkink forcing', this is needed for running an HBV model.

In [ ]:
forcing_path_ERA5 = Path.home() / "forcing" / camels_id / "ERA5"
forcing_path_ERA5.mkdir(exist_ok=True)

In [ ]:
ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
   dataset="ERA5",
   start_time=experiment_start_date,
   end_time=experiment_end_date,
   shape=shapefile,
   directory=forcing_path_ERA5,
)

In [ ]:
print(ERA5_forcing)

So the LumpedMakkinkForcing stores the following:


We can also load the forcing data after we have generated it:

In [ ]:
load_location = forcing_path_ERA5 / "work" / "diagnostic" / "script"  # this is needed because the data is stored in a sub-directory
ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(directory=load_location)